# Shutina Svetlana 381906-2м

# MNIST classification with kNN classifier

In [16]:
import numpy as np
import pandas as pd

## 1. Load MNIST dataset and prepare data

In [17]:
train_data = pd.read_csv("input/mnist_train.csv")
test_data = pd.read_csv("input/mnist_test.csv")

In [18]:
train_data.head()

,label,1x1,1x2,1x3,1x4,1x5,1x6,1x7,1x8,1x9,...,28x19,28x20,28x21,28x22,28x23,28x24,28x25,28x26,28x27,28x28
0,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
x_train, y_train = train_data.drop('label', axis=1).values.astype("uint8"), train_data['label'].values.astype("uint8")
x_test, y_test = test_data.drop('label', axis=1).values.astype("uint8"), test_data['label'].values.astype("uint8")

In [20]:
print("training data points: {}".format(len(x_train)))
print("testing data points: {}".format(len(x_test)))

training data points: 60000
testing data points: 10000


## 2. kNN classifier implementation

In [21]:
import heapq
from heapq import heappop
from heapq import heappush
from heapq import heappushpop

In [25]:
def euclidean_dist(x, y):
    return np.linalg.norm(x - y) 

In [26]:
class Node(object):
    def __init__(self, coords=None, axis=None, left=None, right=None, data=None):
        self.coords = coords
        self.axis = axis
        self.left = left
        self.right = right
        self.data = data

    def __len__(self):
        return len(self.coords)

    def __getitem__(self, i):
        return self.coords[i]

    
class KDTree:
    def __init__(self, X, leaf_size=40, dist=euclidean_dist):
        self.distance = dist
        
        def build_tree(X, idxs, axis=0):
            
            if X is None:
                return None
            
            n = X.shape[0]
            if n > leaf_size:
                median_idx = n // 2
                sort_idxs = np.argpartition(X[:,axis], median_idx)
                X = X[sort_idxs]
                idxs = idxs[sort_idxs]
                node_point = X[median_idx]
                next_axis = (axis + 1) % X.shape[1]

                return Node(node_point, axis, build_tree(X[:median_idx], idxs[:median_idx], next_axis),
                            build_tree(X[median_idx+1:], idxs[median_idx+1:], next_axis), idxs[median_idx])
            return Node(data=(X,idxs))
        
        self.root = build_tree(X.copy(), np.arange(X.shape[0]))     

    def k_nearest_neighbors(self, point, k):

        bests = []      
        
        def search(node):
            
            if node is None:
                return 
            # leaf
            if node.coords is None:
                for x, idx in zip(*node.data):
                    dist = -self.distance(x, point)
                    if len(bests) < k:
                        heappush(bests, (dist, idx))
                    elif dist > bests[0][0]:
                        heappushpop(bests, (dist, idx))
            # not leaf
            else:
                dist = -self.distance(node.coords, point)
                if len(bests) < k:
                    heappush(bests, (dist, node.data))
                elif dist > bests[0][0]:
                    heappushpop(bests, (dist, node.data)) 
                diff = point[node.axis] - node.coords[node.axis]
                sides = (node.left, node.right) if diff < 0 else (node.right, node.left)
                # closer side
                search(sides[0])
                # farther side
                if -abs(diff) > bests[0][0]:
                    search(sides[1])
        
        search(self.root)
        idx_bests = []
        while bests:
            idx_bests = [heappop(bests)[1]] + idx_bests
        return idx_bests    

    def query(self, X, k=1):
        return [self.k_nearest_neighbors(point,k) for point in X]

In [27]:
class KNeighborsClassifier:
    def __init__(self, n_neighbors=5, leaf_size=30, dist=euclidean_dist):
        self.distance = dist
        self.k = n_neighbors
        self.leaf_size = leaf_size
    
    def fit(self, X, y):
#         self.mean = np.mean(X, axis=0)
#         self.std = np.std(X, axis=0)
#         X = (X - self.mean) / self.std
        
        self.kdtree = KDTree(X, self.leaf_size, dist=self.distance)
        self.labels = y
        self.n_classes = np.unique(y).size
        
    def predict_proba(self, X):
#         X = (X - self.mean) / self.std
        predicted_probs = np.zeros(shape=(X.shape[0],self.n_classes))
        for i, point in enumerate(X):
            neighbors = self.kdtree.k_nearest_neighbors(point, self.k)
            for neighbor in neighbors:
                predicted_probs[i][self.labels[neighbor]] += 1
        predicted_probs /= self.k
        
        return predicted_probs
    
    def predict(self, X):
        return np.argmax(self.predict_proba(X), axis=1)

## 3. Check correctness (compare with exist implementations)

In [28]:
from sklearn.metrics import classification_report
from sklearn.neighbors import KNeighborsClassifier as sklearn_KNNclassifier

In [29]:
import sklearn
from sklearn import datasets
if int((sklearn.__version__).split(".")[1]) < 18:
    from sklearn.cross_validation import train_test_split
else:
    from sklearn.model_selection import train_test_split
    
mnist = datasets.load_digits()
 
(trainData, testData, trainLabels, testLabels) = train_test_split(np.array(mnist.data),
mnist.target, test_size=0.25, random_state=42)
 
(trainData, valData, trainLabels, valLabels) = train_test_split(trainData, trainLabels,
test_size=0.1, random_state=84)
 
# show the sizes of each data split
print("training data points: {}".format(len(trainLabels)))
print("validation data points: {}".format(len(valLabels)))
print("testing data points: {}".format(len(testLabels)))

training data points: 1212
validation data points: 135
testing data points: 450


In [30]:
import time
s = time.time()
model = sklearn_KNNclassifier(n_neighbors=1)
model.fit(x_train, y_train)
y_pred = model.predict(x_test)
 
print("EVALUATION ON TESTING DATA")
print(classification_report(y_test, y_pred))
print("time: ", time.time()-s)

s = time.time()
custom_model = KNeighborsClassifier(n_neighbors=10)
custom_model.fit(x_train, y_train)
y_pred2 = custom_model.predict(x_test)

print("EVALUATION ON TESTING DATA")
print(classification_report(y_test, y_pred2))
print("time: ", time.time()-s)

EVALUATION ON TESTING DATA
              precision    recall  f1-score   support

           0       0.98      0.99      0.99       980
           1       0.97      0.99      0.98      1135
           2       0.98      0.96      0.97      1032
           3       0.96      0.96      0.96      1010
           4       0.97      0.96      0.97       982
           5       0.95      0.96      0.96       892
           6       0.98      0.99      0.98       958
           7       0.96      0.96      0.96      1028
           8       0.98      0.94      0.96       974
           9       0.96      0.96      0.96      1009

    accuracy                           0.97     10000
   macro avg       0.97      0.97      0.97     10000
weighted avg       0.97      0.97      0.97     10000

time:  861.0479180812836
EVALUATION ON TESTING DATA
              precision    recall  f1-score   support

           0       1.00      0.48      0.64       980
           1       0.15      1.00      0.27      1135

## 4. Define features and distance functions

In [55]:
from skimage import feature

In [76]:
def getHOGFeature(images, orientations=9, pixelsPerCell=(4, 4), cellsPerBlock=(2, 2), block_norm='L2-Hys'):
    features = []
    for img in images:       
        img = np.reshape(img,(28,28))        
        hogFeature = feature.hog(img, orientations, pixelsPerCell, cellsPerBlock, block_norm) 
        features.append(hogFeature)
    return np.array(features, dtype=np.float32)

In [77]:
hogFeatures_train = getHOGFeature(x_train)
hogFeatures_test = getHOGFeature(x_test)

# distance

In [58]:
import scipy

In [59]:
def manhattan_dist(x, y):
    return scipy.spatial.distance.cityblock(x,y)
def cosine_dist(x, y):
    return scipy.spatial.distance.cosine(x,y)

## 5. Find hyper-parameters

In [60]:
from itertools import product

In [61]:
n_neighbors = [1, 2, 5, 10, 20]
distance = [cosine_dist, manhattan_dist]
params = [n_neighbors, distance]

In [51]:
def get_accuracy(y_pred, y_true):
    t = y_true[y_true == y_pred]
    return t.size / y_true.size

for n,dist in product(*params):
    s = time.time()
    custom_model = KNeighborsClassifier(n_neighbors=n, dist=dist)
    custom_model.fit(x_train, y_train)
    y_pred = custom_model.predict(x_test)
    print(f'n_neighbors={n:<2} dist_function={dist.__name__:<14}\
    nmodel accuracy={get_accuracy(y_test, y_pred):.8}')
    print(f'time: {time.time()-s:.2}')

n_neighbors=1  dist_function=cosine_dist       nmodel accuracy=0.3989
time: 1.1e+02
n_neighbors=1  dist_function=manhattan_dist    nmodel accuracy=0.2547
time: 1e+01
n_neighbors=2  dist_function=cosine_dist       nmodel accuracy=0.3785
time: 5.1e+01
n_neighbors=2  dist_function=manhattan_dist    nmodel accuracy=0.21
time: 1.3e+01
n_neighbors=5  dist_function=cosine_dist       nmodel accuracy=0.4758
time: 2.9e+02
n_neighbors=5  dist_function=manhattan_dist    nmodel accuracy=0.2162
time: 6.6e+02
n_neighbors=10 dist_function=cosine_dist       nmodel accuracy=0.5509
time: 1.2e+02
n_neighbors=10 dist_function=manhattan_dist    nmodel accuracy=0.2054
time: 1.1e+01
n_neighbors=20 dist_function=cosine_dist       nmodel accuracy=0.5947
time: 4.4e+02
n_neighbors=20 dist_function=manhattan_dist    nmodel accuracy=0.1656
time: 2.4e+01


## 6. Results
### The best model parameters:
* k: 10
* Distance function: cosine
* Features: Raw

### The best model test accuracy: 0.5947

###  Why such model parameters are the best?
Your answer...